In [1]:
import tensorflow as tf
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

##### Decoder'un kelime üretmeye başlaması için uyarıcı kelimeye ihtiyacı vardır. Uyarıcı kelimeden sonra üretime başlayacaktır. Uyarıcı kelimenin veri setinde bulunmaması gerekir. Bunun için "baslangicccccccc" kelimesini seçtik. Çevirinin sonlanması için end tokeni de eklememiz lazım.Bu tokeninde veri setinde bulunmaması gerekmekte. Bu yüzden "endddddddddddddddd" kelimesini seçtik. Decoder "baslangicccccccc " ile kelime üretmeye başlayacak " endddddddddddddddd" ile kelime üretimini bitirecek. Buradaki boşluk karakteri önceki veya sonraki kelimeye yapismamasi icindir.

In [3]:
# Bosluk karakteri diger kelimeye yapismamasi icin.
mark_start= "baslangicccccccc "
mark_end= " endddddddddddddddd"

##### data_src encoder kelimeleri, data_dest decoder kelimeleri.

In [4]:
data_src = []
data_dest = []

##### Kullandığımız kaynakta ingilizce ve türkçe metinler /t ile ayrıldığından bu şekilde işleyip kaydediyoruz. Bu veri setini https://www.kaggle.com/datasets/seymasa/turkish-to-english-translation-dataset adresinden bulduk. 

In [5]:
for line in open('/kaggle/input/turkish-to-english-translation-dataset/TR2EN.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')
    
    tr_text = mark_start + tr_text + mark_end
    
    data_src.append(en_text)
    data_dest.append(tr_text)

In [6]:
print(f"eng: {data_src[10000]}\n tr + marks: {data_dest[10000]}")

eng: I like it, too.
 tr + marks: baslangicccccccc Onu ben de beğeniyorum. endddddddddddddddd


In [7]:
# Tokenizer ile metni sayılara çevirir. 
def tokenize_texts(texts, num_words=None):
    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(texts)

    return tokenizer
# Verilen listedeki her öğeyi ters çevirir. 
def reverse_tokens(tokens):
    return [list(reversed(x)) for x in tokens]
# Metini sabit uzunluğa çevirir. 
def pad_tokens(tokens, maxlen, padding, truncating):
    return pad_sequences(tokens, maxlen=maxlen, padding=padding, truncating=truncating)
# Bellek kullanımını sıfırlamak için uzunluğun %95 ini kapsayan metod. 
def calculate_max_tokens(num_tokens):
    return int(np.mean(num_tokens) + 2 * np.std(num_tokens))
# Tokenı kelimeye çevirir. 
def token_to_word(token, index_to_word):
    word=" " if token == 0 else index_to_word[token]
    return word
# Tokenları cümleye çevirir. 
def tokens_to_string(tokens, index_to_word):
    words = [index_to_word[token] for token in tokens if token != 0]
    return " ".join(words)
# Metni token dizisine çevirir. 
def text_to_tokens(text, tokenizer, maxlen, padding, reverse=False):
    tokens = tokenizer.texts_to_sequences([text])
    tokens = np.array(tokens)

    if reverse:
        tokens = np.flip(tokens, axis=1)
        truncating = "pre"
    else:
        truncating = "post"

    return pad_sequences(tokens, maxlen=maxlen, padding=padding, truncating=truncating)
# Verilen metini token dizisine çevirir. 
def tokenize_and_preprocess(texts, padding, reverse=False, num_words=None):
    tokenizer = tokenize_texts(texts, num_words=num_words)
    index_to_word = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))
    
    tokens = tokenizer.texts_to_sequences(texts)

    if reverse:
        tokens = reverse_tokens(tokens)
        truncating = "pre"
    else:
        truncating = "post"

    num_tokens = [len(x) for x in tokens]
    max_tokens = calculate_max_tokens(num_tokens)

    tokens_padded = pad_tokens(tokens, maxlen=max_tokens, padding=padding, truncating=truncating)

    return {
        'tokenizer': tokenizer,
        'index_to_word': index_to_word,
        'tokens': tokens,
        'max_tokens': max_tokens,
        'tokens_padded': tokens_padded,
    }


##### Burada ingilizce kelimeleri tokenize edip ön işliyoruz. Burada prepadding ile düşünce vektörü üretilmeden tüm kelimeler görünüyor ve boyutu eşitlemek için 0 ekliyor. Network en son gördüklerini daha iyi aklında tutacağı için reverse ile ters çeviriliyor.

In [8]:
# Prepadding sayesinde düşünce vektörü üretilmeden tüm kelimeleri görüyor. post olsaydı önce kelimeler daha sonra 0lar ile karşılaşacaktı. Network en son gördüklerini daha iyi aklında tutacağı için prepadding.
# 0lar kelime uzunluğunu eşitlemek için kullanılır. 
tokenizer_src=tokenize_and_preprocess(texts=data_src,padding="pre",reverse=True,num_words=None)

##### Burada çevirinin başında 0larla uğraşmamak için eksik cümlelerin sonuna 0 ekleniyor. 

In [9]:
# Decoderın başlangıçta 0lar ile uğraşmaması için postpadding uygulanır. 
tokenizer_dest=tokenize_and_preprocess(texts=data_dest,padding="post",reverse=False,num_words=None)

##### İleride sık sık kullanacağımız için işlemi kolaylaştırmak adına değişkene atıyoruz. 

In [10]:
tokenizer_dest_word_index = tokenizer_dest["tokenizer"].word_index
tokenizer_src_word_index = tokenizer_src["tokenizer"].word_index

##### Başlangıç Tokenimizi değişkene atıyoruz. 

In [11]:
token_start = tokenizer_dest_word_index.get(mark_start.strip(), None)
token_start

1

##### Bitiş Tokenimizi değişkene atıyoruz. 

In [12]:
token_end = tokenizer_dest_word_index.get(mark_end.strip(), None)
token_end

2

##### İngilizce kelimeler 11 tokenden, türkçe kelimeler 10 tokenden oluşuyor. Eksik token varsa 0 eklenecek, fazla token varsa kesilecek.  

In [13]:
tokens_src = tokenizer_src["tokens_padded"]
tokens_dest = tokenizer_dest["tokens_padded"]
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


##### Cümle kısa olduğundan başına 0 eklenmiş. 

In [14]:
print(tokens_src[2000])

[   0    0    0    0    0    0    0    0 1526  223    3]


In [15]:
print(tokens_to_string(tokenizer_src['tokens_padded'][2000], tokenizer_src['index_to_word']))

weak feel i


##### Cümle uzun olduğundan sonuna 0 eklenmiş. Başlangıç ve Bitiş tokenleri görünüyor. 

In [16]:
print(tokens_dest[2000])

[    1 10140   412     2     0     0     0     0     0     0]


In [17]:
tokens_to_string(tokenizer_dest["tokens_padded"][2000], tokenizer_dest["index_to_word"])

'baslangicccccccc cılız hissediyorum endddddddddddddddd'

## Input State

##### Encoder İçin Input belirlemesi. 

In [18]:
encoder_input_data = tokens_src

##### Decoder için Input belirlemesi. Input ve outputun tek farkı Input outputun 1 kaymış hali

In [19]:
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]

In [20]:
encoder_input_data[2000]

array([   0,    0,    0,    0,    0,    0,    0,    0, 1526,  223,    3],
      dtype=int32)

In [21]:
print(f"Decoder Input: {decoder_input_data[2000]} \nDecoder Output: {decoder_output_data[2000]}")

Decoder Input: [    1 10140   412     2     0     0     0     0     0] 
Decoder Output: [10140   412     2     0     0     0     0     0     0]


##### Burada da kelimeleşmiş halini görüyoruz. 

In [22]:
print(tokens_to_string(decoder_input_data[2000], tokenizer_dest["index_to_word"]))
print(tokens_to_string(decoder_output_data[2000], tokenizer_dest['index_to_word']))

baslangicccccccc cılız hissediyorum endddddddddddddddd
cılız hissediyorum endddddddddddddddd


##### Burada uzunlukları değere atıyoruz. Türkçe daha türetilebilir bir dil olduğundan daha fazla kelime vardır. 

In [23]:
num_encoder_words = len(tokenizer_src_word_index)
num_decoder_words = len(tokenizer_dest_word_index)

print(num_encoder_words)
print(num_decoder_words)

21315
94058


## Encoder

##### Hazır eğitilmiş vektör kullanacağız. Glove ile eğitilmiş kelime vektörleri kullanacağız. Bu adım zorunlu bir adım olmasa da model başarısını arttırabileceği için kullanıyoruz. Kelimeler için 100 uzunluğunda vektörler oluşacak. 

In [24]:
embedding_size = 100

##### Burada glove ile eğitilmiş vektör ile kendi vektörümüzü karşılaştıracağız. Kendi vektörümüzü rastgele değerlerden oluşacak. Eğer glove vektörü ile eşleşirse glove vektörünün değerini alacak. 

In [25]:
word2vec = {}
with open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt', encoding='UTF-8') as f:
    for line in f:
        # Kelime ile vektör boiluk ile ayrıldığı için.
        values = line.split()
        # Önce kelime sonra vektör olduğu için.
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [26]:
# Rastgele değer atama
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size))
for word, i in tokenizer_src_word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        # Eşleşme durumunda eklenmesi. 
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [27]:
embedding_matrix.shape

(21315, 100)

##### Encoder input katmanı

In [28]:
encoder_input = Input(shape=(None,), name='encoder_input')

##### Encoder için embedding katmanı. Topladığımız verileri weights'e atıyoruz ve trainable=true ile eğitilebilir yapıyoruz. Model eğitilirken burası da eğitilecek.

In [29]:
encoder_embedding = Embedding(input_dim=num_encoder_words,
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding')

##### State size belirliyoruz ve daha doğru tahmin için dropout ekliyoruz

In [30]:
state_size = 512
dropout_rate=0.2

In [31]:
encoder_lstm1 = LSTM(state_size,dropout=dropout_rate, name='encoder_lstm1', return_sequences=True)
encoder_lstm2 = LSTM(state_size,dropout=dropout_rate, name='encoder_lstm2', return_sequences=True)
encoder_lstm3 = LSTM(state_size,dropout=dropout_rate, name='encoder_lstm3', return_sequences=True)
encoder_lstm4 = LSTM(state_size,dropout=dropout_rate, name='encoder_lstm4', return_sequences=False)

##### Encoderları birleştiriyoruz

In [32]:
def connect_encoder():
    net = encoder_input
    
    net = encoder_embedding(net)
    
    net = encoder_lstm1(net)
    net = encoder_lstm2(net)
    net = encoder_lstm3(net)
    net = encoder_lstm4(net)
    encoder_output = net
    
    return encoder_output

In [33]:
encoder_output = connect_encoder()

## Decoder

##### Encoderın ürettiği vektör için ve türkçe cümleler için iki input ekliyoruz. 

In [34]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')

In [35]:
decoder_input = Input(shape=(None,), name='decoder_input')

##### Decoder embeddingi için glove vektörü bulamadık. O yüzden kendisi rastgele oluşturup eğitecek.

In [36]:
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [37]:
decoder_lstm1 = LSTM(state_size,dropout=dropout_rate, name='decoder_lstm1', return_sequences=True)
decoder_lstm2 = LSTM(state_size,dropout=dropout_rate, name='decoder_lstm2', return_sequences=True)
# Cümle sequence olduğu için true olmalı
decoder_lstm3 = LSTM(state_size,dropout=dropout_rate, name='decoder_lstm3', return_sequences=True)
decoder_lstm4 = LSTM(state_size,dropout=dropout_rate, name='decoder_lstm4', return_sequences=True)

In [38]:
decoder_dense = Dense(num_decoder_words,
                      activation='linear',
                      name='decoder_output')

##### initial state'i 2 boyutlu vermemizin sebebi lstm yapısından dolayı.

In [39]:
def connect_decoder(initial_state):
    net = decoder_input
    
    net = decoder_embedding(net)
    
    net = decoder_lstm1(net, initial_state=[initial_state, initial_state])
    net = decoder_lstm2(net, initial_state=[initial_state, initial_state])
    net = decoder_lstm3(net, initial_state=[initial_state, initial_state])
    net = decoder_lstm4(net, initial_state=[initial_state, initial_state])

    decoder_output = decoder_dense(net)
    
    return decoder_output

##### Modeli eğitmek için bu modeli oluşturuyoruz. Sadece eğitimde kullanıyoruz.

In [40]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

##### Encoder oluşturuyoruz. Veriler ilk burası ile karşılaşacak ardından vektör üretecek. 

In [41]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

##### Decoder oluşturuyoruz. Encoder ile bağlıyoruz ve encoderdan gelen vektörle kelime üretiyor. 

In [42]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

##### loss fonksiyonu için daha yüksek başarı için özel bir loss fonksiyonu oluşturduk. 

In [43]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

##### Optimizer belirliyoruz. 

In [44]:
optimizer = RMSprop(learning_rate=1e-3)

##### Modeli compile ediyoruz. 

In [45]:
model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy)

##### Burada model için checkpoint oluşturuyoruz. Earlystoppingden daha faydalı olabileceğini düşündüğümüz için bunu kullanıyoruz.

In [46]:
path_checkpoint = 'model.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint,monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)

##### Input ve Outputları belirliyoruz. 

In [47]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}

In [48]:
y_data = {'decoder_output': decoder_output_data}

##### Modeli eğitiyoruz

In [49]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=512,
                epochs=50,
                callbacks=[checkpoint])

Epoch 1/50
924/924 [==============================] - 329s 342ms/step - loss: 4.6456
Epoch 2/50
924/924 [==============================] - 315s 341ms/step - loss: 4.4874
Epoch 3/50
924/924 [==============================] - 315s 341ms/step - loss: 4.2880
Epoch 4/50
924/924 [==============================] - 315s 341ms/step - loss: 4.1009
Epoch 5/50
924/924 [==============================] - 315s 341ms/step - loss: 3.8464
Epoch 6/50
924/924 [==============================] - 315s 341ms/step - loss: 3.5880
Epoch 7/50
924/924 [==============================] - 315s 340ms/step - loss: 3.3326
Epoch 8/50
924/924 [==============================] - 315s 341ms/step - loss: 3.0924
Epoch 9/50
924/924 [==============================] - 315s 340ms/step - loss: 2.8781
Epoch 10/50
924/924 [==============================] - 314s 340ms/step - loss: 2.6948
Epoch 11/50
924/924 [==============================] - 315s 341ms/step - loss: 2.5369
Epoch 12/50
924/924 [==============================] - 315s 341

##### Model encoder ve decoderını kullanmak için kaydediyoruz

In [50]:
model_encoder.save_weights("model_encoder.keras")
model_decoder.save_weights("model_decoder.keras")